In [1]:
import os
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
from concorde.mda import anti_neighbors_vectorized
from concorde.plots import scatter_interpolate
from concorde.tools import ssScale
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
plt.rcParams.update({'font.size': 10})
from matplotlib.offsetbox import AnchoredText
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
# from shapely.geometry import Point
from scipy.stats import gaussian_kde

/tmp/ipykernel_13734/1868100766.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

#### Loaddata

In [3]:
df = pd.read_pickle(r'../data/STORM/processed/batch02/STORM_NA_R5_In_v3_pnt.pkl')
df.head()

,year,month,yr_tc_number,time_step_at_min_press,time_step_at_min_distNC,lat_at_min_press,lon_at_min_press,lat_at_min_distNC,lon_at_min_distNC,min_press_at_min_distNC,...,distNC_at_min_distNC,distNC_at_min_press,heading_dir_at_min_distNC,heading_dir_at_min_press,duration,lat_gen,lon_gen,length,mean_heading_dir_inter,mean_curvature_inter
0,3.0,9.0,1.0,16.0,70.0,16.1,-49.4,33.3,-78.6,997.1,...,86.775640,3383.824337,327.994617,296.565051,219,11.7,-34.3,5.914717e+06,336.801409,0.000000
1,6.0,9.0,10.0,34.0,49.0,22.6,-75.9,34.1,-78.6,931.6,...,59.705246,1269.069372,355.236358,326.309932,162,13.4,-58.8,4.107373e+06,353.744103,0.026774
2,12.0,9.0,2.0,43.0,63.0,28.0,-65.8,35.4,-75.2,963.1,...,54.504938,1252.456827,353.659808,308.659808,210,8.6,-45.7,5.071078e+06,350.055435,0.129835
3,12.0,8.0,8.0,57.0,74.0,28.1,-69.5,35.3,-75.8,991.8,...,11.094700,988.691903,14.036243,296.565051,240,6.8,-30.5,6.057396e+06,13.948292,0.624748
4,16.0,9.0,4.0,2.0,1.0,37.6,-76.4,36.8,-76.1,990.8,...,37.790654,129.925612,339.443955,0.000000,24,35.9,-75.8,7.739767e+05,340.504503,0.000000


In [4]:
len(df)

3626

In [5]:
df['cat_at_min_distNC'] = [ssScale(x) for x in df['max_ws_at_min_distNC'].values]

In [6]:
df = df[df['cat_at_min_distNC'] > 0]
df = df[df['rad_to_max_ws_at_min_distNC'] < df['rad_to_max_ws_at_min_distNC'].quantile(0.995)]
df = df[df['mean_curvature_inter'] < df['mean_curvature_inter'].quantile(0.995)]

df.index = df['tc_number']
dfr = df.drop(['year', 'yr_tc_number', 'tc_number', 'max_cat', 'time_step_at_min_press', 
               'time_step_at_min_distNC', 'lat_at_min_press', 'lon_at_min_press', 
               'min_press_at_min_press', 'max_ws_at_min_press',
               'rad_to_max_ws_at_min_press', 'forward_speed_at_min_press', 
               'distNC_at_min_press', 'heading_dir_at_min_press', 'max_ws_at_min_distNC', 
               'heading_dir_at_min_distNC', 'length', 'distNC_at_min_distNC', 'month', 'cat_at_min_distNC'], 
               axis = 1)

# dfr.to_pickle(r'../data/STORM/processed/batch02/STORM_NA_R5_In_v3_pnt_inpMDA_minCat1.pkl')
dfr.index = range(len(dfr))
print(len(dfr.columns))
dfr.describe().T

11


,count,mean,std,min,25%,50%,75%,max
lat_at_min_distNC,1814.0,34.561742,1.317317,27.000000,33.800000,34.500000,35.300000,39.900000
lon_at_min_distNC,1814.0,-76.861742,1.843054,-84.200000,-78.200000,-76.600000,-75.600000,-69.700000
min_press_at_min_distNC,1814.0,956.767310,15.242113,883.400000,946.800000,959.100000,969.400000,983.900000
rad_to_max_ws_at_min_distNC,1814.0,51.152627,23.481239,9.260000,37.040001,46.299999,60.032873,178.175177
forward_speed_at_min_distNC,1814.0,31.589143,10.457845,3.068625,24.046466,31.513909,38.154161,77.134334
landfall,1814.0,0.954245,0.209011,0.000000,1.000000,1.000000,1.000000,1.000000
duration,1814.0,162.329107,89.188097,33.000000,90.000000,150.000000,216.000000,678.000000
lat_gen,1814.0,18.510033,7.293856,5.200000,12.100000,17.300000,25.200000,36.800000
lon_gen,1814.0,-57.642117,17.089490,-96.800000,-72.775000,-60.100000,-42.800000,-21.100000
mean_heading_dir_inter,1814.0,221.599087,142.385297,0.000000,37.719150,307.709886,332.077007,360.000000


In [7]:
len(dfr)

1814

In [6]:
index = [0 if 'heading_dir' in x else 1 for x in dfr.columns]

In [7]:
cl = anti_neighbors_vectorized(dfr, len(dfr), index)

100%|██████████| 1813/1813 [00:02<00:00, 725.91it/s] 


In [8]:
dfr['tc_number'] = df.index
cl['tc_number'] = df.iloc[cl['index_cluster'], :].index
cl.to_pickle(f'../data/STORM/processed/batch02/STORM_NA_R5_In_v3_pnt_MDAvec_11params_{len(dfr)}_minCat1.pkl')